## Import library

In [1]:
import sys
sys.path.insert(
    1,
    '/home/dtd/Documents/interpretable_machine_learning/Source Code/my_work/lib'
)
sys.path.insert(
    1,
    '/home/dtd/Documents/interpretable_machine_learning/Causal Inference/CEA')

import data_load
import numpy as np
import pandas as pd
import logging
import dowhy.datasets
import dowhy
import propensity_score_estimator as pse
import incremental_ps_score_estimator as ipse
import math
import timeit
import evaluation as evl
import pandas as pd
import numpy as np
import tensorflow as tf
import time

from dowhy import CausalModel
from sklearn.metrics import log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import sem
from dowhy import CausalModel
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import explained_variance_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
from math import sqrt
from econml.dml import LinearDMLCateEstimator
from sklearn.linear_model import LassoCV
from econml.inference import BootstrapInference
from econml.dml import SparseLinearDMLCateEstimator
from sklearn.linear_model import LassoCV, ElasticNetCV
from sklearn.ensemble import GradientBoostingRegressor
from mlens.ensemble import SuperLearner
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
[MLENS] backend: threading


## Load and generate data

In [2]:
file_path = "https://msalicedatapublic.blob.core.windows.net/datasets/Pricing/pricing_sample.csv"
df = pd.read_csv(file_path)

In [3]:
df.head()

,account_age,age,avg_hours,days_visited,friends_count,has_membership,is_US,songs_purchased,income,price,demand
0,3,53,1.834234,2,8,1,1,4.903237,0.960863,1.0,3.917117
1,5,54,7.171411,7,9,0,1,3.330161,0.732487,1.0,11.585706
2,3,33,5.351920,6,9,0,1,3.036203,1.130937,1.0,24.675960
3,2,34,6.723551,0,8,0,1,7.911926,0.929197,1.0,6.361776
4,4,30,2.448247,5,8,1,0,7.148967,0.533527,0.8,12.624123


In [21]:
# Chuẩn bị bloc dữ liệu cho mô hình DML
df['price'] = df['price'].apply(lambda x: 0.85 if x != 1.0 else x)
df['price_bin'] = df['price'].apply(lambda x: 1 if x == 1.0 else 0)

Y = "demand"  # Outcome
T = "price"   # Can thiệp
X = "income"   # Covariates
W = list(df.drop(columns=["demand", "price", "income"]).columns)  # Confounders

In [22]:
def gamma_fn(X):
    return -3 - 14 * (X["income"] < 1)

def beta_fn(X):
    return 20 + 0.5 * (X["avg_hours"]) + 5 * (X["days_visited"] > 4)

def demand_fn(data, T):
    Y = gamma_fn(data) * T + beta_fn(data)
    return Y

def true_te(x, n, stats):
    if x < 1:
        subdata = df[df["income"] < 1].sample(n=n, replace=True)
    else:
        subdata = df[df["income"] >= 1].sample(n=n, replace=True)
    te_array = subdata["price"] * gamma_fn(subdata) / (subdata["demand"])
    if stats == "mean":
        return np.mean(te_array)
    elif stats == "median":
        return np.median(te_array)
    elif isinstance(stats, int):
        return np.percentile(te_array, stats)
    
def revenue_fn(data, discount_level1, discount_level2, baseline_T, policy):
    policy_price = baseline_T * (1 - discount_level1) * policy + baseline_T * (1 - discount_level2) * (1 - policy)
    demand = demand_fn(data, policy_price)
    rev = demand * policy_price
    return rev

In [23]:
#### Re-calculate demand
df['demand'] = demand_fn(df,df['price_bin'].values)

In [50]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.5)

model_t = LogisticRegression()

model_t.fit(train[W].values, train['price_bin'].values)

train['p1'] = 1 - model_t.predict_proba(train[W].values)[:,:1]
train['p0'] = model_t.predict_proba(train[W].values)[:,:1]

test['p1'] = 1 - model_t.predict_proba(test[W].values)[:,:1]
test['p0'] = model_t.predict_proba(test[W].values)[:,:1]


test.head()

/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a co

,account_age,age,avg_hours,days_visited,friends_count,has_membership,is_US,songs_purchased,income,price,demand,price_bin,p1,p0
969,4,48,6.448327,7,11,1,1,7.079105,1.602158,0.85,28.224163,0,0.0,1.0
1462,4,57,3.292444,6,13,0,1,1.644700,0.872394,0.85,26.646222,0,0.0,1.0
503,4,53,4.268072,0,6,1,1,9.995420,0.735982,1.00,5.134036,1,1.0,0.0
7439,5,18,4.556836,0,9,0,1,3.985178,1.274105,0.85,22.278418,0,0.0,1.0
8980,1,19,3.776627,7,10,1,0,9.590984,1.131075,0.85,26.888313,0,0.0,1.0


In [52]:
from collections import Counter
delta_seq = np.linspace(start = 0, stop = 2000, num = 1020)
rev_seq = []
for delta in tqdm(delta_seq):
    q1 = (delta * test['p1']) / (delta * test['p1'] + test['p0'])
    print("Delta ",delta)
    print("Q1 ", q1.values)
    
    policy = np.array([1 if i >= 0.5 else 0 for i in q1])
    print(Counter(policy))
    
    rev = revenue_fn(data = test, 
                     discount_level1 = 0.1, 
                     discount_level2 = 0, 
                     baseline_T = 1, 
                     policy = policy)
    rev_seq.append(np.round(np.mean(rev),2))
    print("Revenue {:.2f} ".format(np.mean(rev)))
    print("*"*100)
    

  3%|▎         | 33/1020 [00:00<00:06, 161.53it/s]

Delta  0.0
Q1  [ 0.  0. nan ...  0.  0.  0.]
Counter({0: 5000})
Revenue 14.41 
****************************************************************************************************
Delta  1.9627085377821394
Q1  [0. 0. 1. ... 0. 0. 0.]
Counter({0: 2794, 1: 2206})
Revenue 13.83 
****************************************************************************************************
Delta  3.9254170755642788
Q1  [0. 0. 1. ... 0. 0. 0.]
Counter({0: 2794, 1: 2206})
Revenue 13.83 
****************************************************************************************************
Delta  5.888125613346418
Q1  [0. 0. 1. ... 0. 0. 0.]
Counter({0: 2794, 1: 2206})
Revenue 13.83 
****************************************************************************************************
Delta  7.8508341511285575
Q1  [0. 0. 1. ... 0. 0. 0.]
Counter({0: 2794, 1: 2206})
Revenue 13.83 
****************************************************************************************************
Delta  9.813542688910697
Q1  

  7%|▋         | 72/1020 [00:00<00:05, 175.10it/s]

Delta  64.7693817468106
Q1  [0. 0. 1. ... 0. 0. 0.]
Counter({0: 2794, 1: 2206})
Revenue 13.83 
****************************************************************************************************
Delta  66.73209028459274
Q1  [0. 0. 1. ... 0. 0. 0.]
Counter({0: 2794, 1: 2206})
Revenue 13.83 
****************************************************************************************************
Delta  68.69479882237488
Q1  [0. 0. 1. ... 0. 0. 0.]
Counter({0: 2794, 1: 2206})
Revenue 13.83 
****************************************************************************************************
Delta  70.65750736015701
Q1  [0. 0. 1. ... 0. 0. 0.]
Counter({0: 2794, 1: 2206})
Revenue 13.83 
****************************************************************************************************
Delta  72.62021589793916
Q1  [0. 0. 1. ... 0. 0. 0.]
Counter({0: 2794, 1: 2206})
Revenue 13.83 
****************************************************************************************************
Delta  74.582924

  9%|▊         | 89/1020 [00:00<00:05, 171.41it/s]

...

Revenue 13.83 
****************************************************************************************************
Delta  1839.0578999018646
Q1  [0. 0. 1. ... 0. 0. 0.]
Counter({0: 2794, 1: 2206})
Revenue 13.83 
****************************************************************************************************
Delta  1841.0206084396468
Q1  [0. 0. 1. ... 0. 0. 0.]
Counter({0: 2794, 1: 2206})
Revenue 13.83 
****************************************************************************************************
Delta  1842.9833169774288
Q1  [0. 0. 1. ... 0. 0. 0.]
Counter({0: 2794, 1: 2206})
Revenue 13.83 
****************************************************************************************************
Delta  1844.946025515211
Q1  [0. 0. 1. ... 0. 0. 0.]
Counter({0: 2794, 1: 2206})
Revenue 13.83 
****************************************************************************************************
Delta  1846.9087340529932
Q1  [0. 0. 1. ... 0. 0. 0.]
Counter({0: 2794, 1: 2206})
Revenue 13.

 97%|█████████▋| 994/1020 [00:06<00:00, 148.85it/s]

Delta  1899.901864573111
Q1  [0. 0. 1. ... 0. 0. 0.]
Counter({0: 2794, 1: 2206})
Revenue 13.83 
****************************************************************************************************
Delta  1901.8645731108932
Q1  [0. 0. 1. ... 0. 0. 0.]
Counter({0: 2794, 1: 2206})
Revenue 13.83 
****************************************************************************************************
Delta  1903.8272816486751
Q1  [0. 0. 1. ... 0. 0. 0.]
Counter({0: 2794, 1: 2206})
Revenue 13.83 
****************************************************************************************************
Delta  1905.7899901864573
Q1  [0. 0. 1. ... 0. 0. 0.]
Counter({0: 2794, 1: 2206})
Revenue 13.83 
****************************************************************************************************
Delta  1907.7526987242395
Q1  [0. 0. 1. ... 0. 0. 0.]
Counter({0: 2794, 1: 2206})
Revenue 13.83 
****************************************************************************************************
Delta  1909

100%|██████████| 1020/1020 [00:06<00:00, 163.18it/s]

Revenue 13.83 
****************************************************************************************************
Delta  1962.7085377821393
Q1  [0. 0. 1. ... 0. 0. 0.]
Counter({0: 2794, 1: 2206})
Revenue 13.83 
****************************************************************************************************
Delta  1964.6712463199215
Q1  [0. 0. 1. ... 0. 0. 0.]
Counter({0: 2794, 1: 2206})
Revenue 13.83 
****************************************************************************************************
Delta  1966.6339548577037
Q1  [0. 0. 1. ... 0. 0. 0.]
Counter({0: 2794, 1: 2206})
Revenue 13.83 
****************************************************************************************************
Delta  1968.5966633954858
Q1  [0. 0. 1. ... 0. 0. 0.]
Counter({0: 2794, 1: 2206})
Revenue 13.83 
****************************************************************************************************
Delta  1970.559371933268
Q1  [0. 0. 1. ... 0. 0. 0.]
Counter({0: 2794, 1: 2206})
Revenue 13.

In [57]:
from collections import Counter
delta_seq = np.linspace(start = 0, stop = 100, num = 1020)
beta_seq = np.linspace(start = 0, stop = 1, num = 10)

rev_seq = []
for delta in tqdm(delta_seq):
    for beta in beta_seq:
        q1 = delta*test['p1'] + beta*test['p0']
        q1 = tf.math.sigmoid(q1)
        policy = np.array([1 if i >= 0.5 else 0 for i in q1])
        print(Counter(policy))


  0%|          | 0/1020 [00:00<?, ?it/s]

Counter({1: 5000})
Counter({1: 5000})
Counter({1: 5000})
Counter({1: 5000})
Counter({1: 5000})
Counter({1: 5000})
Counter({1: 5000})
Counter({1: 5000})
Counter({1: 5000})


  0%|          | 1/1020 [00:04<1:11:39,  4.22s/it]

Counter({1: 5000})
Counter({1: 5000})
Counter({1: 5000})
Counter({1: 5000})
Counter({1: 5000})
Counter({1: 5000})
Counter({1: 5000})
Counter({1: 5000})
Counter({1: 5000})
Counter({1: 5000})


  0%|          | 2/1020 [00:08<1:11:33,  4.22s/it]

...

  0%|          | 4/1020 [00:16<1:11:57,  4.25s/it]

Counter({1: 5000})
Counter({1: 5000})
Counter({1: 5000})
Counter({1: 5000})
Counter({1: 5000})
Counter({1: 5000})


  0%|          | 4/1020 [00:19<1:23:05,  4.91s/it]

Counter({1: 5000})


KeyboardInterrupt: 